# Model Testing for Daily Fantasy Scores

In [5]:
import pandas as pd

from fantasy_py import ContestStyle
from fantasy_py.lineup.strategy import GeneralPrizePool, FiftyFifty


SPORT = 'nhl'
SERVICE = 'draftkings'
STYLE = ContestStyle.CLASSIC
CONTEST_TYPES = GeneralPrizePool

filename = f"{SPORT}-{SERVICE}-{STYLE.name}-{FiftyFifty.NAME}.csv"

df = pd.read_csv(filename)
display(df)

,date,style,type,top_score,last_winning_score,slate_id,team_count,team-med,team-70.0th_pctl,"('med-dfs', '(Unknown)')",...,"('med-dfs', 'D')","('med-dfs', 'G')","('med-dfs', 'LW')","('med-dfs', 'RW')","('70.0th-pctl-dfs', '(Unknown)')","('70.0th-pctl-dfs', 'C')","('70.0th-pctl-dfs', 'D')","('70.0th-pctl-dfs', 'G')","('70.0th-pctl-dfs', 'LW')","('70.0th-pctl-dfs', 'RW')"
0,2018-11-06,classic,FIFTY_FIFTY,73.4,58.0,2805.0,18.0,3.0,4.0,5.6,...,NaN,NaN,NaN,NaN,9.50,NaN,NaN,NaN,NaN,NaN
1,2018-11-06,classic,FIFTY_FIFTY,54.0,41.0,2805.0,18.0,3.0,4.0,5.6,...,NaN,NaN,NaN,NaN,9.50,NaN,NaN,NaN,NaN,NaN
2,2018-11-07,classic,FIFTY_FIFTY,41.9,29.3,2806.0,6.0,2.0,2.5,5.0,...,NaN,NaN,NaN,NaN,8.89,NaN,NaN,NaN,NaN,NaN
3,2018-11-08,classic,FIFTY_FIFTY,51.5,32.7,2807.0,18.0,4.0,4.9,6.0,...,NaN,NaN,NaN,NaN,10.00,NaN,NaN,NaN,NaN,NaN
4,2018-11-09,classic,FIFTY_FIFTY,59.6,37.9,2808.0,12.0,2.0,3.7,4.5,...,NaN,NaN,NaN,NaN,9.30,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,2020-08-31,classic,FIFTY_FIFTY,173.7,131.7,4657.0,4.0,3.0,3.3,NaN,...,9.80,3.50,7.80,6.00,NaN,15.80,11.85,11.41,8.52,8.42
218,2020-09-01,classic,FIFTY_FIFTY,95.3,77.5,4660.0,4.0,2.5,3.1,NaN,...,5.95,4.55,5.80,4.75,NaN,7.88,10.47,10.70,11.70,8.45
219,2020-09-03,classic,FIFTY_FIFTY,144.6,87.6,4664.0,4.0,4.0,4.1,NaN,...,8.75,1.75,10.50,6.00,NaN,10.00,13.45,16.05,13.00,8.30
220,2020-09-03,classic,FIFTY_FIFTY,127.8,85.6,4664.0,4.0,4.0,4.1,NaN,...,8.75,1.75,10.50,6.00,NaN,10.00,13.45,16.05,13.00,8.30
